# NARM+




In [11]:
#Importing functions.
import pandas
import csv
from collections import Counter, OrderedDict, defaultdict, namedtuple
import itertools
import math
import numpy as np
import pandas
import random
from sklearn.model_selection import train_test_split
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# PyTorch can run on CPU or on Nvidia GPU (video card) using CUDA
# This cell selects the GPU if one is available.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.synchronize() 

In [12]:
pandas.__version__

'0.24.2'

In [13]:
# Create sessions
sample = pandas.read_pickle("./data/processedData.pkl")
sample['SESSION'] = pandas.to_datetime(sample['TIMESTAMP'],unit='s').dt.date

print(len(sample["USERID"].unique()))
print(len(sample["PRODUCTID"].unique()))


11832
67172


In [14]:
print("Average number of sessions per user")
print(sample.groupby('USERID')['SESSION'].nunique().mean())

print("Average number of clicks per session")
sample.groupby(['USERID', 'SESSION'])['ACTION'].count().mean()

Average number of sessions per user
5.765128465179175
Average number of clicks per session


11.017386715142274

In [15]:
#DATASET CREATION

# read CSV file
# sample = pandas.read_csv("./data/UserBehavior.csv", names=["USERID", "PRODUCTID", "CATEGORYID", "ACTION", "TIMESTAMP"])

# edit size and print size of userID
# sample.set_index("USERID")
# sampleNew = sample[sample["USERID"] < 10319]
# print(len(sampleNew["USERID"].unique()))
# print(len(sampleNew["PRODUCTID"].unique()))

#sampleNew.to_pickle('./data/datasetName.pkl')

In [16]:
# ### DO NOT RUN THIS CELL IF YOU READ THE PICKLE processedData

# ##Preprocessing the Data
# #Remove products that occur less than 5 times
# productsize = sample.groupby(["PRODUCTID"]).size() > 4
# indexProducts = productsize.index[productsize]
# productTotal = sample[sample["PRODUCTID"].isin(indexProducts)]

# #Remove sessions that have a length of 1 or less
# #Remove users that have 2 or less sessions
# sessionList = productTotal.groupby(["USERID","SESSION"]).size() > 1
# sessionBase = sessionList.to_frame(name = "realSESSION").reset_index()
# enoughSessions = sessionBase.groupby(["USERID", "realSESSION"]).size() > 2
# trueSessionBase = enoughSessions.to_frame(name = "enoughSESSIONS").reset_index()


# #Merge DataSets such that original Dataset is restored with newly added filters
# totalDataSet = productTotal.merge(trueSessionBase)
# totalDataSet = totalDataSet[totalDataSet["enoughSESSIONS"] == True]

# #Drop useless columns
# totalDataSet = totalDataSet.drop("enoughSESSIONS", axis=1)
# totalDataSet = totalDataSet.drop("realSESSION", axis=1)
# sample = totalDataSet

# #Piece of mind stuff
# userList = totalDataSet["USERID"].unique()
# productList = totalDataSet["PRODUCTID"].unique()
# print(len(totalDataSet2))
# print(len(userList))
# print(len(productList))


# sample.to_pickle("./data/processedData.pkl")

In [19]:
userList = sample["USERID"].unique()
productList = sample["PRODUCTID"].unique()

In [17]:
# Here we first define a class that can map a product to an ID (p2i)
# and back (i2p).

class OrderedCounter(Counter, OrderedDict):
    """Counter that remembers the order elements are first seen"""
    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__,
                      OrderedDict(self))
    def __reduce__(self):
        return self.__class__, (OrderedDict(self),)


class Vocabulary:
    """A vocabulary, assigns IDs to tokens"""
    def __init__(self):
        self.freqs = OrderedCounter()
        self.users = []
        self.u2i = {}
        self.i2u = []
        self.p2i = {}
        self.i2p = []

    def count_product(self, t):
        self.freqs[t] += 1
    
    def count_user(self, t):
        self.users.append(t)

    def add_product(self, t):
        self.p2i[t] = len(self.p2i)
        self.i2p.append(t) 
        
    def add_user(self, t):
        self.u2i[t] = len(self.u2i)
        self.i2u.append(t)

    def build(self, min_freq=0):
        self.add_product("<unk>")  # reserve 0 for <unk> (unknown products (products only occuring in test set))
        self.add_user("<unk>")
        tok_freq = list(self.freqs.items())
        tok_freq.sort(key=lambda x: x[1], reverse=True)
        for tok, freq in tok_freq:
            if freq >= min_freq:
                self.add_product(tok)
        for user in self.users:
            self.add_user(user)

In [20]:
# This process should be deterministic and should have the same result 
# if run multiple times on the same data set.

def build_voc(userList, productList):
    v = Vocabulary()
    for product in productList:
        v.count_product(product)
    for user in userList:
        v.count_user(user)
    v.build()
    return v

v = build_voc(userList, productList)
print("Vocabulary size:", len(v.p2i))



Vocabulary size: 67173


In [21]:
# Create nested list of sessions and items per user
userBase = sample.groupby(['USERID', 'SESSION'])['PRODUCTID'].apply(list).groupby('USERID').apply(list)
print(userBase[1])

[[2268318, 2333346], [4365585, 230380], [2951368, 3108797], [2734026, 4152983, 266784, 266784, 1305059], [2087357, 3157558], [2087357, 1340922, 4954999], [3219016, 2028434, 3219016], [4954999, 818610, 271696]]


In [22]:
# More efficient create examples function
# A simple way to define a class is using namedtuple.
Example = namedtuple("Example", ["userID", "history", "inputs", "target"])

def f(userid, sessions, train):
    #print(sessions)
    non_empty_sessions = [ses for ses in sessions if len(ses) >= 2]
    if len(non_empty_sessions) < 3:
        print('Not enough sessions')
        return
    if train:
        object_train = Example(userID = userid, history = 
                               [item for sublist in non_empty_sessions[:-2] for item in sublist], 
                               inputs = non_empty_sessions[-2][:-1], target = non_empty_sessions[-2][1:])
        return object_train
    else:
        return Example(userID = userid, history = 
                       [item for sublist in non_empty_sessions[:-1] for item in sublist], 
                       inputs = non_empty_sessions[-1][:-1], 
                       target = non_empty_sessions[-1][1:])

def createExamples(userBase):
    ''' Create training and testing set '''
    userBase = pandas.DataFrame(userBase)
    userBase.reset_index(level = 0, inplace = True)
    trainData = [x for x in 
                 userBase.apply(lambda x: f(x['USERID'], x['PRODUCTID'], True), axis = 1).tolist() 
                 if x is not None]
    testData = [x for x in 
                userBase.apply(lambda x: f(x['USERID'], x['PRODUCTID'], False), axis = 1).tolist() 
                if x is not None]
    return trainData, testData

trainData, testData = createExamples(userBase)
print(trainData[0])
print('')
print(testData[0])
    
    

Example(userID=1, history=[2268318, 2333346, 4365585, 230380, 2951368, 3108797, 2734026, 4152983, 266784, 266784, 1305059, 2087357, 3157558, 2087357, 1340922, 4954999], inputs=[3219016, 2028434], target=[2028434, 3219016])

Example(userID=1, history=[2268318, 2333346, 4365585, 230380, 2951368, 3108797, 2734026, 4152983, 266784, 266784, 1305059, 2087357, 3157558, 2087357, 1340922, 4954999, 3219016, 2028434, 3219016], inputs=[4954999, 818610], target=[818610, 271696])


In [23]:
# HELPER FUNCTIONS

# function to yield one example at a time
def get_examples(data, shuffle=True, **kwargs):
    """Shuffle data set and return 1 example at a time (until nothing left)"""
    if shuffle:
#         print("Shuffling training data")
        random.shuffle(data)  # shuffle training data each epoch
    for example in data:
        yield example
    
# function to prepare an example for usage by the model
def prepare_example(example, vocab):
    """
    Turn an example into tensors of inputs and target.
    """
    u = vocab.u2i.get(example.userID,0)
    v = torch.LongTensor([u])
    v = v.to(device)
    
    w = torch.LongTensor([vocab.p2i.get(t, 0) for t in example.history])
    w = w.to(device)
    
    x = torch.LongTensor([vocab.p2i.get(t, 0) for t in example.inputs])[:,None]
    x = x.to(device)

    y = torch.LongTensor([vocab.p2i.get(t, 0) for t in example.target])[:,None]
    y = y.to(device)

    return (v, w, x), y

# simple evaluation function
def simple_evaluate(model, data, prep_fn=prepare_example, **kwargs):
    """Precision of a model on given data set."""
    model.eval()  # disable dropout
    targets = []
    predictions = []
    
    vocab = model.vocab
    for example in data:
        # convert the example input and targets to PyTorch tensors
        targets.extend([vocab.p2i.get(t,0) for t in example.target])
        x, target = prepare_example(example, vocab)

        # forward pass
        # get the output from the neural network for input x
        with torch.no_grad():
            output, alphas = model(x)
        # output shape: (sequence length, score for each item)
        prediction = torch.argmax(output, dim=1).tolist()
        predictions.extend(prediction)
            
    precision = sum([1 if p==t else 0 for p,t in zip(predictions,targets)])/len(targets)

    return precision, None

def recall(model, data, prep_fn=prepare_example, at=5, **kwargs):
    model.eval() # disable dropout
    targets = []
    predictions = []
    
    vocab = model.vocab
    for example in data:
        # convert the example input and targets to PyTorch tensors
        targets.extend([vocab.p2i.get(t,0) for t in example.target])
        x, target = prepare_example(example, vocab)

        # forward pass
        # get the output from the neural network for input x
        with torch.no_grad():
            output, alphas = model(x)
        # output shape: (sequence length, score for each item)
        prediction = torch.argsort(output, dim=1, descending=True)[:,:at].tolist()
        predictions.extend(prediction)
        
    recall = sum([1 if t in p else 0 for t,p in zip(targets,predictions)])/len(targets)
    
    return recall, None

def mrr(model, data, prep_fn=prepare_example, at=5, **kwargs):
    model.eval() # disable dropout
    targets = []
    predictions = []
    
    vocab = model.vocab
    for example in data:
        # convert the example input and targets to PyTorch tensors
        targets.extend([vocab.p2i.get(t,0) for t in example.target])
        x, target = prepare_example(example, vocab)

        # forward pass
        # get the output from the neural network for input x
        with torch.no_grad():
            output, alphas = model(x)
        # output shape: (sequence length, nr of products)
        prediction = torch.argsort(output, dim=1, descending=True)[:,:at].tolist()
        predictions.extend(prediction)
        
    mrr = sum([1/(p.index(t) + 1) if (t != 0 and t in p) else 0 for t,p in zip(targets,predictions)])/len(targets)
    
    return mrr, None

In [24]:
# Custom NN

class NarmPlus(nn.Module):
    def __init__(self, 
                 item_embedding_dim, user_embedding_dim, hidden_size, output_dim, num_layers, 
                 vocab, 
                 activation_fn=nn.RReLU(), dropout=0.2):
        super(NarmPlus, self).__init__()
        # Store parameters
        self.item_embedding_dim = item_embedding_dim
        self.user_embedding_dim = user_embedding_dim
        self.hidden_size = hidden_size # hidden size is also user embedding dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        # Shape of hidden_state: (num_layers * num_directions, batch, hidden_size)
        self.hidden_state_dim = (num_layers, 1, hidden_size)
        self.hidden_state_size = num_layers * hidden_size
        self.vocab = vocab
        num_users = len(vocab.u2i)
        num_items = len(vocab.p2i)
        
        # General part
        self.ActivationFn = activation_fn
        self.Softmax = nn.Softmax(dim=1)
        self.loss = self.top1loss
        self.dropout = nn.Dropout(p=dropout)
        
        # History part
        self.UserEmbedding = nn.Embedding(num_users, user_embedding_dim)
        self.ItemEmbedding = nn.Embedding(num_items, item_embedding_dim)
        self.LatentItemHistory = nn.Linear(item_embedding_dim, user_embedding_dim)
        self.ProfileToHidden = nn.Linear(user_embedding_dim, self.hidden_state_size)
        
        # NARM Part
        # Input to the GRU is the item embedding: input_size = embedding_size
        # Hidden size is something we can experiment with
        self.Local = nn.GRU(item_embedding_dim, hidden_size, num_layers=num_layers, batch_first=True)
        self.Global = nn.GRU(item_embedding_dim, hidden_size, num_layers=num_layers, batch_first=True)
        self.Decoder = nn.Bilinear(item_embedding_dim, 2*hidden_size, output_dim)
        
        # Inner working of NARM attention part
        # Latent space for alpha: what value to pick?
        # I assume no bias, based on the paper
        latent_space = hidden_size
        self.A1 = nn.Linear(hidden_size,latent_space,bias=False)
        self.A2 = nn.Linear(hidden_size,latent_space,bias=False)
        self.v = nn.Linear(latent_space,1,bias=False)
        
        
    def forward(self,x):
        user, history, inputs = x
        # user shape (1)
        # history shape (history_length)
        # inputs shape (seq_length,1)
        item_embeds_h = self.dropout(self.ItemEmbedding(history))
#         print(f'Item embedding shape (history): {item_embeds_h.shape}')
        # item_embeds_h shape (history_length, embedding_size)
        user_embed = self.dropout(self.UserEmbedding(user))
#         print(f'User embedding shape: {user_embed.shape}')
        # user_embed shape (1, embedding_size) <- shouldn't this be (1, hidden_state_size)?
        dense = self.LatentItemHistory(item_embeds_h)
        dense = self.ActivationFn(dense)
#         print(f'Dense shape: {dense.shape}')
        # dense shape (history_length, hidden_state_size)
        alpha1 = self.Softmax(torch.matmul(user_embed,torch.transpose(dense, 0, 1)))
#         print(f'Alpha1 shape: {alpha1.shape}')
        # alpha shape (history_length)
        profile = torch.sum(torch.mul(alpha1,torch.transpose(dense,0,1)),1)
#         print(f'Profile shape: {profile.shape}')
        # profile shape (hidden_state_size)
        # reshape to correct hidden state dimensions
        h_0 = torch.reshape(self.ProfileToHidden(profile),self.hidden_state_dim)
#         print(f'h_0 shape: {h_0.shape}')
        
        # Embed the items of the current session
        item_embeds_c = self.dropout(torch.transpose(self.ItemEmbedding(inputs),0,1))
#         print(f'Item embedding shape (current): {item_embeds_c.shape}')
        # shape (seq_length, embedding_size)
        # add a batch dimension to the front, necessary for GRU
#         item_embeds_c = item_embeds_c[None,:,:] 
#         print(f'Item embedding shape (current): {item_embeds_c.shape}')
        
        out_local, _ = self.Local(item_embeds_c,h_0)
#         print(f'Out_local shape: {out_local.shape}')
        out_global, _ = self.Global(item_embeds_c,h_0)
#         print(f'Out_global shape: {out_global.shape}')
        # out shape (batch_size, seq_length, hidden_size), containing hidden state output for every step
        # Shape of c_global and c_local should be: (sequence, hidden_size)
        c_global = out_global[0,:,:]
#         print(f'c_global shape: {c_global.shape}')
        # c_global shape (seq_length, hidden_size)
        c_local, alphas = self.calculate_c_local(out_local)
#         print(f'c_local shape: {c_local.shape}')
        
        # Shape of c_global and c_local should be: (sequence, hidden_size)
        c = self.dropout(torch.cat((c_global, c_local), dim=1))
        # Decoder takes as inputs: embeddings for each item and c
        embeds = self.ItemEmbedding(torch.LongTensor([i for i in range(len(self.vocab.p2i))]).to(device))
        # Make embeds and c the same shape
        embeds = embeds.repeat(c.shape[0],1,1).contiguous()
        c = c[:,None,:].repeat(1,embeds.shape[1],1).contiguous()
#         print(f'c shape: {c.shape}')
#         print(f'All item embeds shape: {embeds.shape}')
        out = self.Decoder(embeds, c)[:,:,0]
#         print(f'Output shape: {out.shape}')
        output = self.Softmax(out)
        return output, alphas
    
    def calculate_c_local(self,H):
        # H: hidden states returned from the GRU
        # H shape (batch, sequence, hidden size)
        H = H[0,:,:]
        # H shape (sequence, hidden size)
        # Initialise c_local with the output hidden states: every hidden state has a similarity of 
        # 1 with itself, so the entire hidden state is taken into account
        c_local_base = H.clone().detach().requires_grad_(True)
        
  
        alphas = torch.ones((H.shape[0],H.shape[0]),dtype=torch.float32).to(device)
        cs = [torch.zeros((1,H.shape[1]), dtype=torch.float32).to(device)]
        for t in range(H.shape[0]):
            # If it is the first hidden state, then there are no previous hidden states to calculate the 
            # alpha from and the current hidden state has already been saved in c_local.
            if t == 0:
                continue
            
            # Technically we do not need to store the alphas, but maybe we want to do something with these values
            alphas_t = torch.zeros(H.shape[0], dtype=torch.float32).to(device)
            A1 = self.A1(H[t])
            for j in range(t):
                A2 = self.A2(H[j])
                # next three lines could be done in one line
                alphas_t[j] = self.v(self.ActivationFn(A1 + A2))
#                 ct_j = torch.mul(alphas_t[j],H[j])
#                 c_local[t] = torch.add(c_local[t],ct_j)
            ct = torch.sum(alphas_t[:t].unsqueeze(1) * H[:t,:],dim=0).unsqueeze(0)
            cs.append(ct)
            alphas[t] = alphas_t
        c_local = torch.cat(cs,dim=0) + c_local_base
        return c_local, alphas
    
    def top1(self, yhat):
        ''' Top1 loss, yhat is vector with softmax probabilities '''
        # Not sure if you can just call backward to this, but I think it should work. Code from:
        # https://github.com/mquad/hgru4rec/blob/master/src/hgru4rec.py
        yhatT = torch.transpose(yhat, 0, 1)
        loss = torch.mean(torch.mean(nn.sigmoid( - torch.diag(yhat) + yhatT) + nn.sigmoid(yhat ** 2), dim = 0) 
                          - nn.sigmoid(T.diag(yhat ** 2)))
        return loss
    
    def top1loss(self, output, targets):
        scores_for_targets = torch.gather(output, 1, targets)
        loss = torch.mean(torch.sigmoid(output - scores_for_targets) +
                torch.sigmoid(output**2))
        return loss
        

In [32]:
# function to train a model
name_extension = ''
def train_model(model, optimizer, num_epochs=200, 
                print_every=1, eval_every=1,
                batch_fn=get_examples, 
                prep_fn=prepare_example,
                eval_fn=simple_evaluate,
                batch_size=5, eval_batch_size=None,
                predict=False
               ):
    """Train a model."""  
    train_loss = 0.
    start = time.time()
    best_eval = 0.
    best_iter = 0
    criterion=model.loss

    # store train loss and validation accuracy during training
    # so we can plot them afterwards
    losses = []
    accuracies = []  

    if eval_batch_size is None:
        eval_batch_size = batch_size
    
    vocab = model.vocab
    
    for epoch in range(num_epochs):
        batch_iter = 0
        batch_out = []
        batch_targets = []
        for example in batch_fn(train_data): # goes through the entire training data once, a.k.a. an epoch

            # forward pass, make sure the model is in train modus
            model.train()
            x, targets = prep_fn(example, vocab)
            output, alphas = model(x)
            # output shape (sequence length, nr of products): a score for each product at each time step
            # alphas are the alphas used in the Narm part

            if len(batch_out) == 0:
                batch_out = output
                batch_targets = targets
            else:
                batch_out = torch.cat((batch_out, output), dim=0)
                batch_targets = torch.cat((batch_targets, targets), dim=0)
                
            batch_iter += 1
                
            if batch_iter == batch_size:
                # enough users put through the network, time to calculate the loss and update the network
                
                # calculate loss
                # important: len(batch_out) is not always the same
                # it does always contain data on the same number of users
                # batch_size, in other words, is how many users to look at before updating
                loss = criterion(batch_out, batch_targets)
                train_loss += loss.item()
                
                # backward pass
                # erase previous gradients
                model.zero_grad()

                # compute gradients
                loss.backward()

                # update weights - take a small step in the opposite dir of the gradient
                optimizer.step()
                
                # reset the iterator, batch_out and batch_targets
                batch_iter = 0
                batch_out = []
                batch_targets = []
                
        if (epoch + 1) % print_every == 0:
            print("Epoch %r: loss=%.4f, time=%.2fs" % 
                 (epoch + 1, train_loss, time.time()-start))
            losses.append(train_loss)       
            train_loss = 0.
            
        if (epoch + 1) % eval_every == 0:
            accuracy, _ = eval_fn(model, dev_data, batch_size=eval_batch_size,
                                         batch_fn=batch_fn, prep_fn=prep_fn)
            accuracies.append(accuracy)
            print("epoch %r: dev acc=%.4f" % (epoch + 1, accuracy))       

            # save best model parameters
            if accuracy > best_eval:
                print("new highscore")
                best_eval = accuracy
                best_iter = epoch + 1
                path = "{}{}.pt".format(model.__class__.__name__,name_extension)
                ckpt = {
                  "state_dict": model.state_dict(),
                  "optimizer_state_dict": optimizer.state_dict(),
                  "best_eval": best_eval,
                  "best_iter": best_iter
                }
                torch.save(ckpt, path)
    
    # Done training
    # evaluate on train, dev, and test with best model
    print("Loading best model")
    path = "{}{}.pt".format(model.__class__.__name__,name_extension)        
    ckpt = torch.load(path)
    model.load_state_dict(ckpt["state_dict"])

    train_acc, _ = eval_fn(
        model, train_data, batch_size=eval_batch_size, 
        batch_fn=batch_fn, prep_fn=prep_fn)
    dev_acc, _ = eval_fn(
        model, dev_data, batch_size=eval_batch_size,
        batch_fn=batch_fn, prep_fn=prep_fn)
    test_acc, predictions = eval_fn(
        model, test_data, batch_size=eval_batch_size, 
        batch_fn=batch_fn, prep_fn=prep_fn)

    print("best model iter {:d}: "
          "train acc={:.4f}, dev acc={:.4f}, test acc={:.4f}".format(
              best_iter, train_acc, dev_acc, test_acc))

    return test_acc, predictions

In [26]:
def createSplits(data, k):
    folds = {}
    for i in range(k):
        dev = data[math.ceil(i*len(data)/k) : math.ceil((i+1)*len(data)/k)]
        train = [x for x in data if x not in dev]
        folds[i] = train,dev
    return folds

In [ ]:
if device == 'cuda':
    torch.cuda.empty_cache()
device = 'cpu'

# item_embedding_dim, hidden_size, output_dim, num_layers, vocab, 
train_data = trainData
dev_data = test_data = testData

num_users = len(userList)
num_products = len(productList)

model = NarmPlus(math.ceil(num_products**0.25),math.ceil(num_users**0.25),10,1,1,v,dropout=0.2)
model.to(device)
optimizer = optim.Adam(model.parameters())
a, p = train_model(model, optimizer, eval_fn=recall, num_epochs=10)

In [28]:
a = torch.tensor([i for i in range(12)]).reshape((4,3))
print(a)
b = torch.tensor([[1],[0],[2],[0]])
b = torch.t(torch.tensor([[1,0,2,0]]))
print(torch.gather(a, 1, b))
print(a - torch.gather(a, 1, b))
print(a.shape, b.shape)
print(a.tolist())

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])
tensor([[1],
        [3],
        [8],
        [9]])
tensor([[-1,  0,  1],
        [ 0,  1,  2],
        [-2, -1,  0],
        [ 0,  1,  2]])
torch.Size([4, 3]) torch.Size([4, 1])
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]


NameError: name 'ceil' is not defined

In [ ]:
def top1loss(output, targets):
    scores_for_targets = torch.gather(output, 1, targets)
    loss = nn.Mean(nn.Sigmoid(output - scores_for_targets) +
            nn.Sigmoid(output**2))
    return loss
    